# Анализ пассажиров Титаника

## Описание данных

Датасет `titanic.csv`, содержащит информацию о пассажирах легендарного лайнера "Титаник", затонувшего в ночь с 14 на 15 апреля 1912 года после столкновения с айсбергом. 

Датасет включает демографические данные пассажиров, информацию о билетах и главное — данные о выживаемости.

**Датасет**: Titanic Dataset  
**Источник**: Stanford CS109  
**Прямая ссылка**: https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv  
**Лицензия**: Public Domain


### Столбцы датасета

| Столбец | Тип данных | Описание | Пример значений |
|---------|-----------|----------|-----------------|
| **Survived** | int (0/1) | Выжил ли пассажир | 0 = Нет, 1 = Да |
| **Pclass** | int (1-3) | Класс пассажира | 1 = Первый, 2 = Второй, 3 = Третий |
| **Name** | string | Полное имя пассажира | "Braund, Mr. Owen Harris" |
| **Sex** | string | Пол пассажира | "male" / "female" |
| **Age** | float | Возраст в годах | 22.0, 38.0, 26.0 |
| **Siblings/Spouses Aboard** | int | Количество братьев/сестер/супругов на борту | 0, 1, 2, ... |
| **Parents/Children Aboard** | int | Количество родителей/детей на борту | 0, 1, 2, ... |
| **Fare** | float | Стоимость билета в фунтах стерлингов | 7.25, 71.28, 0.0 |


## ИМПОРТ БИБЛИОТЕК

In [ ]:
import pandas as pd
import numpy as np

## ЗАГРУЗКА И ПЕРВИЧНЫЙ АНАЛИЗ

In [ ]:
# 1. Загрузка датасета
url = 'https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv'
df = pd.read_csv(url)

In [ ]:
# 2. Первые 10 строк
df.head(10)

In [ ]:
# 3. Информация о структуре данных
df.info()

In [ ]:
# 4. Описательная статистика
df.describe()

In [ ]:
# 5. Список всех колонок
df.columns.tolist()

In [ ]:
# 6. Размер датасета
df.shape
print(f"Строк: {df.shape[0]}, Столбцов: {df.shape[1]}")

## ОБРАБОТКА ПРОПУСКОВ И ДУБЛИКАТОВ

In [ ]:
# 7. Количество пропущенных значений
df.isnull().sum()

In [ ]:
# 8. Процент пропусков
(df.isnull().sum() / len(df) * 100).round(2)

In [ ]:
# 9. Заполнение пропусков в Age медианой
df['Age'].fillna(df['Age'].median(), inplace=True)

In [ ]:
# 10. Проверка полных дубликатов
duplicates = df.duplicated().sum()
print(f"Количество полных дубликатов: {duplicates}")

In [ ]:
# 11. Удаление строк с пропущенным Fare
df.dropna(subset=['Fare'], inplace=True)

In [ ]:
# 12. Проверка оставшихся пропусков
df.isnull().sum()

## ПРЕОБРАЗОВАНИЕ ТИПОВ ДАННЫХ

In [ ]:
# 13. Survived в булевый тип
df['Survived'] = df['Survived'].astype(bool)

In [ ]:
# 14. Age в float
df['Age'] = df['Age'].astype(float)

In [ ]:
# 15. Pclass в категориальный тип
df['Pclass'] = df['Pclass'].astype('category')

In [ ]:
# Проверка типов данных
df.dtypes

## СОЗДАНИЕ НОВЫХ ПРИЗНАКОВ

In [ ]:
# 16. family_size
df['family_size'] = df['Siblings/Spouses Aboard'] + df['Parents/Children Aboard']

In [ ]:
# 17. is_alone
df['is_alone'] = df['family_size'] == 0

In [ ]:
# 18. age_group
def categorize_age(age):
    if age < 18:
        return 'child'
    elif age <= 60:
        return 'adult'
    else:
        return 'senior'

df['age_group'] = df['Age'].apply(categorize_age)

In [ ]:
# 19. fare_category
def categorize_fare(fare):
    if fare < 10:
        return 'cheap'
    elif fare <= 30:
        return 'medium'
    else:
        return 'expensive'

df['fare_category'] = df['Fare'].apply(categorize_fare)

In [ ]:
# 20. Извлечение титула из имени
df['title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)

In [ ]:
# 21. Первые 10 строк с новыми столбцами
df.head(10)

## ФИЛЬТРАЦИЯ И СОРТИРОВКА ДАННЫХ

In [ ]:
# 22-23. Пассажиры первого класса
first_class = df[df['Pclass'] == 1]
print(f"Количество пассажиров первого класса: {len(first_class)}")

In [ ]:
# 24. Выжившие женщины
survived_women = df[(df['Survived'] == True) & (df['Sex'] == 'female')]
print(f"Количество выживших женщин: {len(survived_women)}")

In [ ]:
# 25. Query: 3 класс, возраст < 18
young_third_class = df.query('Pclass == 3 and Age < 18')
print(f"Количество: {len(young_third_class)}")

In [ ]:
# 26. Сортировка по Fare (убывание)
sorted_by_fare = df.sort_values('Fare', ascending=False)
sorted_by_fare[['Name', 'Fare', 'Pclass']].head(10)

In [ ]:
# 27. Сортировка по Pclass, затем Age
sorted_multi = df.sort_values(['Pclass', 'Age'])
sorted_multi[['Name', 'Pclass', 'Age']].head(10)

In [ ]:
# 28. Топ-20 самых дорогих билетов
top_20_expensive = df.nlargest(20, 'Fare')
top_20_expensive[['Name', 'Fare', 'Pclass']]

## ГРУППИРОВКА И АГРЕГАЦИЯ

In [ ]:
# 29. Количество пассажиров по классам
passengers_by_class = df.groupby('Pclass').size()
passengers_by_class

In [ ]:
# 30. Процент выживших по классам
survival_by_class = df.groupby('Pclass')['Survived'].mean() * 100
survival_by_class.round(2)

In [ ]:
# 31. Класс с наибольшей выживаемостью
best_class = survival_by_class.idxmax()
print(f"Класс с наибольшей выживаемостью: {best_class}, Выживаемость: {survival_by_class[best_class]:.2f}%")

In [ ]:
# 32. Группировка по полу
sex_stats = df.groupby('Sex').agg({
    'Name': 'count',
    'Age': 'mean',
    'Survived': 'mean'
})
sex_stats.columns = ['Количество', 'Средний возраст', 'Процент выживших']
sex_stats['Процент выживших'] *= 100

sex_stats.round(2)

In [ ]:
# 33. Агрегация по классам
class_agg = df.groupby('Pclass').agg({
    'Age': ['mean', 'min', 'max'],
    'Fare': ['mean', 'median'],
    'Survived': 'mean'
})

class_agg.round(2)

In [ ]:
# 34. Топ-5 титулов по количеству выживших
title_survivors = df[df['Survived'] == True].groupby('title').size().nlargest(5)

title_survivors

## СВОДНЫЕ ТАБЛИЦЫ

In [ ]:
# 35. Сводная таблица: Pclass × Sex, процент выживших
pivot_class_sex = pd.pivot_table(
    df,
    values='Survived',
    index='Pclass',
    columns='Sex',
    aggfunc='mean'
) * 100

pivot_class_sex.round(2)

In [ ]:
# 36. Средний Fare: age_group × Pclass
pivot_age_class = pd.pivot_table(
    df,
    values='Fare',
    index='age_group',
    columns='Pclass',
    aggfunc='mean'
)

pivot_age_class.round(2)

In [ ]:
# 37. С итоговыми значениями
pivot_with_margins = pd.pivot_table(
    df,
    values='Fare',
    index='age_group',
    columns='Pclass',
    aggfunc='mean',
    margins=True
)

pivot_with_margins.round(2)

In [ ]:
# 38. Crosstab для Pclass и Survived
crosstab_result = pd.crosstab(df['Pclass'], df['Survived'])

crosstab_result

## ОБЪЕДИНЕНИЕ ДАТАФРЕЙМОВ

In [ ]:
# 39. Датафрейм со статистикой по классам
class_stats = df.groupby('Pclass').agg({
    'Survived': 'mean',
    'Age': 'mean',
    'Fare': 'mean'
}).reset_index()

class_stats.columns = ['Pclass', 'survival_rate', 'avg_age', 'avg_fare']

In [ ]:
# 40. Статистика по возрастным группам
survival_by_age = df.groupby('age_group').agg({
    'Survived': 'mean',
    'Name': 'count'
}).reset_index()

survival_by_age.columns = ['age_group', 'survival_rate', 'count']

In [ ]:
# 41. Left join с class_stats ДАТАСЕТ С ДОБАВЛЕННОЙ СТАТИСТИКОЙ ПО КЛАССАМ
df_merged = df.merge(class_stats, on='Pclass', how='left', suffixes=('', '_class'))

In [ ]:
# 42. Разделение на выживших и невыживших
survived = df[df['Survived'] == True]
not_survived = df[df['Survived'] == False]

In [ ]:
# 43. Объединение обратно
df_concat = pd.concat([survived, not_survived], ignore_index=True)
print(f"Размер объединенного датасета: {df_concat.shape}")

## СОХРАНЕНИЕ РЕЗУЛЬТАТОВ

In [ ]:
# 44. Сохранение в CSV
df.to_csv('titanic_clean.csv', index=False)

In [ ]:
# 45. Сохранение в Excel с несколькими листами
with pd.ExcelWriter('titanic_analysis.xlsx', engine='openpyxl') as writer:
    # Лист 1: Очищенные данные
    df.to_excel(writer, sheet_name='Clean_Data', index=False)
    
    # Лист 2: Описательная статистика
    df.describe().to_excel(writer, sheet_name='Statistics')
    
    # Лист 3: Группировка по классам
    class_analysis = df.groupby('Pclass').agg({
        'Survived': 'mean',
        'Age': 'mean',
        'Fare': ['mean', 'median', 'max'],
        'Name': 'count'
    })
    class_analysis.to_excel(writer, sheet_name='By_Class')

        
    # Лист 4: Группировка по полу
    sex_analysis = df.groupby('Sex').agg({
        'Survived': 'mean',
        'Age': 'mean',
        'Fare': 'mean',
        'Name': 'count'
    })
    sex_analysis.to_excel(writer, sheet_name='By_Sex')